In [1]:
#load explore and plot data
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Train test split
from sklearn.model_selection import train_test_split

#text pre-processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

#modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional

In [2]:
!pip install wordcloud

In [3]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [4]:
df1 = pd.read_csv("train.En.csv")
df = df1.drop(["Unnamed: 0", "rephrase", "sarcasm", "irony", "satire", "understatement", "overstatement", "rhetorical_question"] , axis=1)

df.head()

,tweet,sarcastic
0,The only thing I got from college is a caffein...,1
1,I love it when professors draw a big question ...,1
2,Remember the hundred emails from companies whe...,1
3,Today my pop-pop told me I was not “forced” to...,1
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1


In [5]:
df.describe()

,sarcastic
count,3468.000000
mean,0.250000
std,0.433075
min,0.000000
25%,0.000000
50%,0.000000
75%,0.250000
max,1.000000


In [6]:
df.groupby('sarcastic').describe().T

sarcastic                                                     0  \
tweet count                                                2600   
      unique                                               2594   
      top     @AsdaServiceTeam imagine your delivery being 2...   
      freq                                                    3   

sarcastic                                                     1  
tweet count                                                 867  
      unique                                                867  
      top     The only thing I got from college is a caffein...  
      freq                                                    1

ham_msg_cloud = WordCloud(width =520, height =260, stopwords = STOPWORDS, max_font_size = 50, background_color = "black", colormap = 'Pastel1').generate(ham_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(ham_msg_cloud, interpolation = 'bilinear')
plt.axis('off') # turn off axis
plt.show()

spam_msg_cloud = WordCloud(width =520,
                          height =260,
                          stopwords=STOPWORDS,
                          max_font_size=50, 
                          background_color ="black",
                         colormap='Pastel1').generate(spam_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(spam_msg_cloud, interpolation='bilinear')
plt.axis('off') # turn off axis
plt.show()

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
tweets= list(df['tweet'])
# Function to preprocess tweets
def preprocess_tweets(tweets):
    stop_words = set(stopwords.words('english'))
    preprocessed_tweets = []
    for tweet in tweets:
        # Tokenize the tweet
        words = word_tokenize(tweet)
        # Remove stopwords and lowercase words
        words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
        preprocessed_tweets.extend(words)
    return preprocessed_tweets


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chloe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chloe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Assuming you have a DataFrame called df with columns 'tweet' and 'sarcastic' representing your data
sarcastic_tweets = df[df['sarcastic'] == 1]['tweet']
non_sarcastic_tweets = df[df['sarcastic'] == 0]['tweet']


In [14]:
sarcastic_tweets

0      The only thing I got from college is a caffein...
1      I love it when professors draw a big question ...
2      Remember the hundred emails from companies whe...
3      Today my pop-pop told me I was not “forced” to...
4      @VolphanCarol @littlewhitty @mysticalmanatee I...
                             ...                        
862               yo @claires do yall do hysterectomies?
863    @JacobWohlReport Do I need to aquire a wife be...
864    I get a lot of boy who cried wolf vibes from t...
865    Update: holding hands with your mom and walkin...
866    I might be rubbish at driving, and have a less...
Name: tweet, Length: 867, dtype: object

In [17]:
list(sarcastic_tweets)

['The only thing I got from college is a caffeine addiction',
 'I love it when professors draw a big question mark next to my answer on an exam because I’m always like yeah I don’t either ¯\\_(ツ)_/¯',
 'Remember the hundred emails from companies when Covid started getting real? I’ve gotten three in regards to support for protests. And only @SavageXFenty shared helpful links and actually said black lives matter... we love capitalism 🥰🙌🏼',
 'Today my pop-pop told me I was not “forced” to go to college 🙃 okay sure sureeee',
 '@VolphanCarol @littlewhitty @mysticalmanatee I did too, and I also reported Cancun Cruz not worrying about the heartbeats of his constituents without electricity or heat when he fled to Mexico.',
 '@jimrossignol I choose to interpret it as "XD": the universal emoticon for laughing at those poor, poor folks in Ubisoft\'s marketing department who have to deal with that branding until the servers quietly shut down 8 months after launch.',
 "Why would Alexa's recipe for 

In [ ]:

# Preprocess sarcastic tweets
sarcastic_text = preprocess_tweets(sarcastic_tweets)

# Preprocess non-sarcastic tweets
non_sarcastic_text = preprocess_tweets(non_sarcastic_tweets)


In [18]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Assuming you have preprocessed text for sarcastic and non-sarcastic tweets and stored them in variables sarcastic_text and non_sarcastic_text

# Generate word cloud for sarcastic tweets
sarcastic_cloud = WordCloud(width=520, height=260, stopwords=STOPWORDS, max_font_size=50, background_color="black", colormap='Pastel1').generate(list(sarcastic_text))
plt.figure(figsize=(16, 10))
plt.imshow(sarcastic_cloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Sarcastic Tweets')
plt.show()

# Generate word cloud for non-sarcastic tweets
non_sarcastic_cloud = WordCloud(width=520, height=260, stopwords=STOPWORDS, max_font_size=50, background_color="black", colormap='Pastel1').generate(list(non_sarcastic_text))
plt.figure(figsize=(16, 10))
plt.imshow(non_sarcastic_cloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Non-Sarcastic Tweets')
plt.show()


TypeError: expected string or bytes-like object